In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pingouin as pg
from utils import *


# Directory where the data files are located
directory = '../data/ADNI/data_normalized/'
file_prefix = 'data_'
file_suffix = '.csv'

functions = [
    # pearson_correlation,
    # cosine_similarity,
    # partial_correlation,
    # correlations_correlation,
    # associated_high_order_fc,
    # euclidean_distance,
    # knn_graph,
    # spearman_correlation,
    # kendall_correlation,
    mutual_information,
    # cross_correlation,
    # granger_causality,
]

# Iterate through files from 1 to 1306

for function in functions:
    for i in range(1, 1306):
        file_path = os.path.join(directory, f"{file_prefix}{i}{file_suffix}")
        
        # Check if the file exists
        if not os.path.exists(file_path):
            print(f"File {file_path} does not exist. Skipping.")
            continue
        
        # Read the data
        data = pd.read_csv(file_path, header=None)
        
        # Calculate the  correlation matrix



        # corr_matrix = mutual_information(data)
        corr_matrix = function(data)

        
        corr_df = pd.DataFrame(corr_matrix)

        # plot_correlation_matrix(corr_df, f"Correlation matrix for {file_path}")

        # visualize the correlation matrix

        # get the name of the function in str
        function_name = function.__name__

        # save the correlation matrix
        # save the name of the file
        path_of_folder = f'../data/ADNI/fmri_edge/{function_name}/'
        if not os.path.exists(path_of_folder):
            os.makedirs(path_of_folder)

        corr_df.to_csv(path_of_folder + f"{function_name}{i}{file_suffix}", index=False)


        print(f"Processed file {file_path}")

print("Processing complete.")

Dynamic graph


In [ ]:
correlation_dyn_graphs = []
for function in functions:
    for i in range(1, 1306):
        dynamic_correlation_matrixes = []
        # calculate the dynamic graph metrics
        time_window = 50
        stride = 3
        for i in range(0, data.shape[0] - time_window, stride):
            window_data = data[i:i+time_window]
            correlation_matrix = function(window_data)
            dynamic_correlation_matrixes.append(correlation_matrix)
        # Convert each dynamic Pearson correlation matrix into a Data object
        data_list = [correlation_matrix_to_graph_data(matrix) for matrix in dynamic_correlation_matrixes]
        data_batch = Batch.from_data_list(data_list)
        correlation_dyn_graphs.append(data_batch)
        function_name = function.__name__
        path_of_folder = f'../data/ADNI/fmri_edge/{function_name}_dynamic/'
        if not os.path.exists(path_of_folder):
            os.makedirs(path_of_folder)
        torch.save(data_batch, path_of_folder + f"{function_name}_dynamic{i}.pt")
        
